In [6]:
import numpy as np
import pandas as pd

date1 = pd.to_datetime('20220820')
date2 = pd.to_datetime('20220821')
(date1 - date2).days

-1

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import warnings
import execjs
import datetime
import time
from FundDataCrawler import FundDataCrawler


warnings.filterwarnings('ignore')
plt.rcParams['font.family'] = 'SimHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 8)

endDate = '20220820'
startDate = '20210802'

def getData(start, end):
    '''
    获取指定时间范围内数据
    '''
    data = fundData.drop(fundData[fundData.index < pd.to_datetime(start)].index, axis=0, inplace=False)
    data.drop(data[data.index > pd.to_datetime(end)].index, axis=0, inplace=True)
    return data

def mean_days(df, days, wave):
    '''
     获取n日均线,最近m日振幅
    :param df: 数据集
    :param days: 均线天数
    :params wave:最近m日振幅
    :return average,wavelength:均线、近m日振幅
    '''
    average = []
    wavelength = []
    start = df.head(1).index-np.timedelta64(days+1, "D")
    start = start.astype('str')[0].replace("-", "")  # 转换为字符串

    df4 = getData(start, endDate)
    for i in df3.index:
        start_date = i-np.timedelta64(days+1, "D")
        ave = df4[(df4.index > start_date) & (df4.index < i)]['收盘价'].mean()
        average.append(ave)
    ########求振幅#######
    start2 = df.head(1).index-np.timedelta64(wave+1, "D")
    start2 = start2.astype('str')[0].replace("-", "")  # 转换为字符串
    df5 = getData(start2, endDate)

    for i in df3.index:
        start_date = i-np.timedelta64(wave+1, "D")
        interval = df5[(df5.index > start_date) & (df5.index < i)]['收盘价']
        length = interval.max()/interval.min()-1  # 最大涨跌幅
        wavelength.append(length)

    return average, wavelength


def stratege(ml, wl, T_1):
    '''
    定投策略
    :param ml:均线
    :param wl:振幅
    :param T_1:前1日收盘价
    '''
    cal = T_1/ml-1  # 大于0,则高于均线
    if (cal >= 0 and cal < 0.15):
        return 0.9
    elif (cal > 0.15 and cal < 0.5):
        return 0.8
    elif (cal >= 0.5 and cal < 1):
        return 0.7
    elif (cal >= 1):
        return 0.6

    elif (wl > 0.05):

        if (cal >= -0.05 and cal < 0):
            return 0.6
        elif (cal >= -0.1 and cal < -0.05):
            return 0.7
        elif (cal >= -0.2 and cal < -0.1):
            return 0.8
        elif (cal >= -0.3 and cal < -0.2):
            return 0.9
        elif (cal >= -0.4 and cal < -0.3):
            return 1.0
        elif (cal < -0.4):
            return 1.1
    else:
        if (cal >= -0.05 and cal < 0):
            return 1.8
        elif (cal >= -0.1 and cal < -0.05):
            return 1.9
        elif (cal >= -0.2 and cal < -0.1):
            return 2.0
        elif (cal >= -0.3 and cal < -0.2):
            return 2.1
        elif (cal >= -0.4 and cal < -0.3):
            return 2.2
        elif (cal < -0.4):
            return 2.3


def smart_invest(df1, frequence, invest_money, start_time, days, wave, balance):
    '''
     定投计算
    :param df1: 数据集
    :param frequence: 定投频率
    :param invest_money: 每次定投金额
    :param start: 定投起始日期
    :param days: 参考均线天数

    :return (amount,invest_log): (收益数据DataFrame，定投记录dict)
    '''
    invest_log = {}  # 每次定投的日期记录(日期:大盘指数)
    invest_day = start_time  # 每次投资的时间
    invest_amount = 0  # 总投资金额
    profile = 0  # 总投资收益
    amount = 0  # 账户总资产

    profile_log = []  # 总收益日志
    invest_amount_log = []  # 账户投资金额日志
    amount_log = []  # 总资产日志
    Yield = []  # 收益率日志

    res = mean_days(df1, days, wave)
    df1["均线"] = res[0]  # 获取均线
    df1["振幅"] = res[1]  # 获取振幅
    for date, quote_change, index, ml, wl in zip(df1.index, df1['涨跌幅'], df1['收盘价'], df1["均线"], df1["振幅"]):

        profile += quote_change*amount  # 计算当天收益率
        profile_log.append(profile)

        # 判断是否为定投日
        if date == invest_day:
            if (invest_day == start_time):
                T_1 = start_time
            else:
                formal_day = 1
                while (True):
                    T_1 = date-np.timedelta64(formal_day, "D")  # 前1天的收盘价
                    if (T_1 in df1.index.tolist()):
                        break
                    else:
                        formal_day += 1
            T_1 = df1[df1.index == T_1]["收盘价"][0]
            rate = stratege(ml, wl, T_1) - balance
            invest_amount += invest_money*rate  # 定投
            invest_log[invest_day] = index  # 记录定投当日的指数

            # 判断7天后是否为交易日,如果不是则往后加1天直到找到交易日
            invest_day += np.timedelta64(frequence, 'D')
            flag = 0
            while (True):
                if (df1[df1.index == invest_day].index == invest_day):
                    break
                else:
                    invest_day += np.timedelta64(1, 'D')
                    flag += 1
                    if (flag == 100):
                        break
        
        invest_amount_log.append(invest_amount)
        amount = invest_amount+profile  # 更新账户总资产
        amount_log.append(amount)
        try:
            Yield.append(profile/amount*100)  # 更新收益率
        except:
            Yield.append(0)
    print("总资产：", amount)
    print("总收益：", profile)
    print("收益率: ", profile/amount*100, "%")

    over = pd.DataFrame({
        "日期": df1.index,
        "收益率": Yield,
        "账户资产": amount_log,
        "投资金额": invest_amount_log
    })
    over = over.set_index("日期")
    return over, invest_log


def myplot(df1, res, buy, titlename):
    '''
    绘制定投结果图
    '''
    plt.figure()
    df1['收盘价'].plot(label="大盘指数")
    plt.scatter(buy.keys(), buy.values(),
                color="brown", marker=".", label="定投记录")
    plt.legend(loc='best')
    plt.ylabel("指数")
    plt.twinx()

    res['账户资产'].plot(color="red")
    res['投资金额'].plot(color="orange")
    plt.ylabel("元")
    plt.legend()
    plt.title(titlename+":{:.2f}%".format(res.tail(1)["收益率"][0]))
    plt.show()


fundDataCrawler = FundDataCrawler()
fundData = fundDataCrawler.getFund('161725')
df3 = getData(startDate, endDate)
total = 10000
frequence = 9
wave = 10
days = 500
balance = 10
for balance in range(90,110,1):
    realDays = (pd.to_datetime(endDate) - pd.to_datetime(startDate)).days
    invest_money = total / (realDays / frequence)  # 每次定投金额
    print("定投频率：{}日，参考均线天数：{}，振幅：{}，每次定投：{}，参数比率：{}".format(frequence,days,wave,invest_money,balance/100))
    res3, buy3 = smart_invest(df3, frequence, invest_money, pd.to_datetime(startDate), days, wave, balance/100)
    # myplot(df3, res3, buy3, "定投频率：{}日，每次定投：{}，收益率：".format(frequence,invest_money))

定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.9
总资产： 2781.915114796003
总收益： -14.429532723579193
收益率:  -0.5186906188055025 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.91
总资产： 2688.4111733597165
总收益： -11.58882664028277
收益率:  -0.43106600489984476 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.92
总资产： 2594.9072319234315
总收益： -8.748120556986434
收益率:  -0.33712652419185085 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.93
总资产： 2501.403290487144
总收益： -5.907414473690289
收益率:  -0.23616401626064182 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.94
总资产： 2407.899349050859
总收益： -3.0667083903940306
收益率:  -0.1273603230800681 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.95
总资产： 2314.3954076145733
总收益： -0.22600230709780034
收益率:  -0.009765068940001868 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率：0.96
总资产： 2220.8914661782874
总收益： 2.6147037761986
收益率:  0.11773217268910421 %
定投频率：9日，参考均线天数：500，振幅：10，每次定投：234.98694516971278，参数比率

In [23]:
for i in range(1,10):
    print(i/10)

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
